# Toronto Neighborhood Clustering

_For Coursera Capstone week 3 Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto_

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 2>

1. <a href="#item1">Wrangle and Clean Toronto Neighborhood Data</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Cluster Neighborhoods in Toronto</a>

4. <a href="#item4">Examine Clusters</a>    
</font>
</div>

## 1. Wrangle and Clean Toronto Neighborhood Data

The data for clustering Toronto's neighborhoods will be sourced by wikipedia. This dataset is indexed by postal code and needs to be scrubbed for unassigned zones (ex: large companies like Amazon may secure a unique postal code for large-volume shipping, etc).

In [1]:
wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

import pandas as pd
import io
import requests

html_content=requests.get(wiki_url).content
df = pd.read_html(html_content)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [2]:
# Rename columns
df.columns = ['PostalCode', 'Borough', 'Neighborhood']
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
# Ignore cells with a borough that is 'Not assigned'
unassigned_boroughs_indeces = df[df['Borough'] == 'Not assigned'].index
df.drop(unassigned_boroughs_indeces, inplace=True)
df

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [4]:
# Examine dataframe for 'Not assigned' neighborhoods
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
7,M7A,Queen's Park,Not assigned


In [5]:
# Name 'Not assigned' neighborhoods after borough name
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']

# Expect empty mask after conditional replacement
df[df['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood


In [6]:
# df.groupby(['PostalCode', 'Borough'], as_index=False)[['Neighborhood']].agg([('Neighborhood', ', '.join)])
formatted_df = df.groupby(['PostalCode', 'Borough'], as_index=False).agg({'Neighborhood': lambda x: ', '.join(x)})
formatted_df.reset_index()
formatted_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [7]:
formatted_df.shape

(103, 3)

### Combine wikipedia data with geocoder data

Since Wikipedia dataset does not include zip code coordinates, we should hydrate dataset with longitude and latitude from Geocoder in order to access Foursquare data.

**Instead of following instructions tightly** I've elected to move forward with the original dataset grouped by neighborhood. If the goal is to eventually compare neighbhorhoods, we should use their coordinates intead of the coordinates of a block of zip codes.

In [42]:
df.describe()

,PostalCode,Borough,Neighborhood
count,210,210,210
unique,103,11,207
top,M8Y,Etobicoke,Queen's Park
freq,8,44,2


In [47]:
# We expected all Neighborhood values to be unique
# In fact, to do the same analysis as NYC, we do not need postal code at all
df.drop("PostalCode", axis=1, inplace=True)
df.drop_duplicates(inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210 entries, 2 to 285
Data columns (total 2 columns):
Borough         210 non-null object
Neighborhood    210 non-null object
dtypes: object(2)
memory usage: 14.9+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 2 to 285
Data columns (total 2 columns):
Borough         208 non-null object
Neighborhood    208 non-null object
dtypes: object(2)
memory usage: 4.9+ KB
None


In [29]:
# Hydrate dataset with geospatial data
# import sys
# !conda install --yes --prefix {sys.prefix} -c conda-forge geopandas
# !conda install --yes --prefix {sys.prefix} -c conda-forge geopy

In [38]:
from geopy.geocoders import Nominatim

locator = Nominatim(user_agent="toronto_geocoder")

# test locator with sample neighborhood name
location = locator.geocode("Rouge, Toronto, Ontario")
print(location)

Rouge, Scarborough—Rouge Park, Scarborough, Toronto, Ontario, M1B 2K5, Canada


In [84]:
from geopy.extra.rate_limiter import RateLimiter

geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# use .loc for column addition to avoid SettingWithCopyWarning
df.loc[:, 'Location'] = df['Neighborhood'].apply(lambda neigh: geocode("{}, Toronto, Ontario".format(neigh)) if neigh else None)
df.loc[:, 'Point'] = df['Location'].apply(lambda loc: tuple(loc.point) if loc else None)

df.loc[:, 'Latitude'] = df['Point'].apply(lambda t: t[0] if t else None)
df.loc[:, 'Longitude'] = df['Point'].apply(lambda t: t[1] if t else None)

df.drop(["Location", "Point"], axis=1, inplace=True)
df

,Borough,Neighborhood,latitude,longitude
2,North York,Parkwoods,43.758800,-79.320197
3,North York,Victoria Village,43.732658,-79.311189
4,Downtown Toronto,Harbourfront,43.640080,-79.380150
5,North York,Lawrence Heights,43.722778,-79.450933
6,North York,Lawrence Manor,43.722079,-79.437507
...,...,...,...,...
281,Etobicoke,Kingsway Park South West,43.650352,-79.500009
282,Etobicoke,Mimico NW,43.616677,-79.496805
283,Etobicoke,The Queensway West,43.623618,-79.514764
284,Etobicoke,Royal York South West,43.648183,-79.511296


In [89]:
# Examine our new columns
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208 entries, 2 to 285
Data columns (total 4 columns):
Borough         208 non-null object
Neighborhood    208 non-null object
latitude        198 non-null float64
longitude       198 non-null float64
dtypes: float64(2), object(2)
memory usage: 8.1+ KB


In [93]:
df[df['latitude'].isnull()]

,Borough,Neighborhood,latitude,longitude
34,York,Humewood-Cedarvale,NaN,NaN
48,York,Caledonia-Fairbanks,NaN,NaN
94,North York,CFB Toronto,NaN,NaN
131,York,Del Ray,NaN,NaN
173,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN
220,Downtown Toronto,Railway Lands,NaN,NaN
224,Etobicoke,Humber Bay Shores,NaN,NaN
228,Etobicoke,Beaumond Heights,NaN,NaN
239,Downtown Toronto,Stn A PO Boxes 25 The Esplanade,NaN,NaN
264,East Toronto,Business Reply Mail Processing Centre 969 Eastern,NaN,NaN


In [97]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df

,Borough,Neighborhood,latitude,longitude
0,North York,Parkwoods,43.758800,-79.320197
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,North York,Lawrence Heights,43.722778,-79.450933
4,North York,Lawrence Manor,43.722079,-79.437507
...,...,...,...,...
193,Etobicoke,Kingsway Park South West,43.650352,-79.500009
194,Etobicoke,Mimico NW,43.616677,-79.496805
195,Etobicoke,The Queensway West,43.623618,-79.514764
196,Etobicoke,Royal York South West,43.648183,-79.511296


In [106]:
#Visualize our neighborhoods
# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

address = 'Toronto, Ontario'

location = geocode(address)
latitude = location.latitude
longitude = location.longitude

map_neighborhoods = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['latitude'], df['longitude'], df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neighborhoods)  
    
map_neighborhoods

The geograpical coordinate of Toronto are 43.653963, -79.387207.


## 2. Explore Neighborhoods in Toronto

In order to draw comparisons with NYC data, we should cluster neighborhoods.
Unlike our NYC analysis, let's cluster across all boroughs instead of across one borough.

#### Define Foursquare Credentials and Version

In [111]:
CLIENT_ID = '5YWYDD1OYF3I3QYJMUFJX3O1UDV5MU4NCBI30FYK4YHMGWQ4' # your Foursquare ID
CLIENT_SECRET = 'LFUHV1IDE4KESRNAONHO1YX42CCC5GICZ112WKKL3HHJ2PGU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Define Foursquare `/explore` helper

In [116]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue',  
                  'Venue Category']
    
    return(nearby_venues)

In [132]:
venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Lawrence Manor
Queen's Park
Rouge
Malvern
Don Mills North
Woodbine Gardens
Parkview Hill
Ryerson
Garden District
Glencairn
Cloverdale
Islington
Martin Grove
Princess Gardens
West Deane Park
Highland Creek
Rouge Hill
Port Union
Flemingdon Park
Don Mills South
Woodbine Heights
St. James Town
Bloordale Gardens
Eringate
Markland Wood
Old Burnhamthorpe
Guildwood
Morningside
West Hill
The Beaches
Berczy Park
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor
Downsview North
Wilson Heights
Thorncliffe Park
Adelaide
King
Richmond
Dovercourt Village
Dufferin
Scarborough Village
Fairview
Henry Farm
Oriole
Northwood Park
York University
East Toronto
Harbourfront East
Toronto Islands
Union Station
Little Portugal
Trinity
East Birchmount Park
Ionview
Kennedy Park
Bayview Village
Downsview East
The Danforth West
Riverdale
Design Exchange
Toronto Dominion Centre
Brockton
Exhibition Place
Parkdale Village
Clair

In [134]:
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
venues_onehot.loc[:, 'Neighborhood'] = venues['Neighborhood'] 

neighborhood_venues = venues_onehot.groupby(["Neighborhood"], as_index=False).mean()
neighborhood_venues

,Neighborhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Adelaide,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.000000,0.00,0.000000,0.0,0.01,0.0,0.000000,0.00,0.000000
1,Agincourt,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.00,0.090909,0.0,0.00,0.0,0.000000,0.00,0.000000
2,Agincourt North,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.035714,0.00,0.000000,0.0,0.00,0.0,0.035714,0.00,0.000000
3,Albion Gardens,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000
4,Alderwood,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Woodbine Heights,0.111111,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000
192,York Mills,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000
193,York Mills West,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.000000
194,York University,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.000000,0.00,0.000000,0.0,0.00,0.0,0.000000,0.00,0.034483


### Find most frequent venue types per neighborhood

In [135]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [137]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhood_top_venues = pd.DataFrame(columns=columns)
neighborhood_top_venues['Neighborhood'] = neighborhood_venues['Neighborhood']

for ind in np.arange(neighborhood_venues.shape[0]):
    neighborhood_top_venues.iloc[ind, 1:] = return_most_common_venues(neighborhood_venues.iloc[ind, :], num_top_venues)

neighborhood_top_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Italian Restaurant,Hotel,Cosmetics Shop,Gastropub,Clothing Store,American Restaurant,Art Gallery
1,Agincourt,Chinese Restaurant,Train Station,Food Court,Vietnamese Restaurant,Korean Restaurant,Asian Restaurant,Coffee Shop,Hong Kong Restaurant,Yoga Studio,Ethiopian Restaurant
2,Agincourt North,Ice Cream Shop,Chinese Restaurant,Bakery,Indian Restaurant,Pizza Place,Taco Place,Beer Store,Sporting Goods Shop,Spa,Sandwich Place
3,Albion Gardens,Beer Store,Fast Food Restaurant,Pizza Place,Fried Chicken Joint,Sandwich Place,Pharmacy,Hardware Store,Grocery Store,Ethiopian Restaurant,Event Space
4,Alderwood,Pizza Place,Pharmacy,Skating Rink,Sandwich Place,Coffee Shop,Pub,Gym,Pool,Farmers Market,Empanada Restaurant


In [138]:
neighborhood_top_venues.shape

(196, 11)

## 3. Cluster neighborhoods

Run k-means to cluster neighborhoods into 5 clusters.

In [140]:
from sklearn.cluster import KMeans

# set number of clusters
k = 5

neighborhood_venues_clustering = neighborhood_venues.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k, random_state=0).fit(neighborhood_venues_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 4, 4, 4, 4, 0, 4, 4, 3, 4], dtype=int32)

In [153]:
# add clustering labels
neighborhood_top_venues.append(pd.DataFrame(kmeans.labels_, columns=["Cluster Labels"]), sort=False)

clustered_neighborhoods = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
clustered_neighborhoods = clustered_neighborhoods.join(neighborhood_top_venues.set_index('Neighborhood'), on='Neighborhood', how="inner")
clustered_neighborhoods.head()

,Borough,Neighborhood,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.758800,-79.320197,4,Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Liquor Store,Gas Station,Discount Store,Bus Line,Convenience Store,Pharmacy,Laundry Service
1,North York,Victoria Village,43.732658,-79.311189,0,Mediterranean Restaurant,Thai Restaurant,Bus Line,Middle Eastern Restaurant,Yoga Studio,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market
2,Downtown Toronto,Harbourfront,43.640080,-79.380150,4,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Pizza Place,History Museum,Music Venue,Sporting Goods Shop,Sports Bar
3,North York,Lawrence Heights,43.722778,-79.450933,4,Clothing Store,Coffee Shop,Cosmetics Shop,American Restaurant,Restaurant,Jewelry Store,Toy / Game Store,Fast Food Restaurant,Accessories Store,Bookstore
4,North York,Lawrence Manor,43.722079,-79.437507,0,Electronics Store,Bank,Park,Pharmacy,Kids Store,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant


In [166]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(clustered_neighborhoods['latitude'], clustered_neighborhoods['longitude'], clustered_neighborhoods['Neighborhood'], clustered_neighborhoods['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
       
map_clusters

## 4. Examine Clusters

### Cluster 0 - Gas Stations, Metro Stations, Baseball Fields

In [158]:
clustered_neighborhoods.loc[clustered_neighborhoods['Cluster Labels'] == 0, clustered_neighborhoods.columns[[1] + list(range(5, clustered_neighborhoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Mediterranean Restaurant,Thai Restaurant,Bus Line,Middle Eastern Restaurant,Yoga Studio,Field,Exhibit,Falafel Restaurant,Farm,Farmers Market
4,Lawrence Manor,Electronics Store,Bank,Park,Pharmacy,Kids Store,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant
20,Rouge Hill,Train Station,Bus Line,Yoga Studio,Field,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
24,Woodbine Heights,Skating Rink,ATM,Athletics & Sports,Park,Pharmacy,Dance Studio,Bus Stop,Cosmetics Shop,Airport Lounge,Ethiopian Restaurant
28,Markland Wood,Baseball Field,Golf Course,Piano Bar,Park,Yoga Studio,Festival,Event Space,Exhibit,Falafel Restaurant,Farm
29,Old Burnhamthorpe,Dog Run,Flower Shop,Gas Station,Park,Yoga Studio,Festival,Event Space,Exhibit,Falafel Restaurant,Farm
30,Guildwood,Train Station,Baseball Field,Storage Facility,Yoga Studio,Field,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
36,Leaside,Convenience Store,Sandwich Place,Park,Japanese Restaurant,Home Service,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit
41,Bathurst Manor,Playground,Convenience Store,Baseball Field,Park,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm
42,Downsview North,Metro Station,Gas Station,French Restaurant,Park,Playground,Furniture / Home Store,Bus Station,Gym Pool,Coffee Shop,Gym / Fitness Center


### Cluster 1 - Parks, Yoga Studios, Event Spaces

In [159]:
clustered_neighborhoods.loc[clustered_neighborhoods['Cluster Labels'] == 1, clustered_neighborhoods.columns[[1] + list(range(5, clustered_neighborhoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Rouge,Park,Fast Food Restaurant,Yoga Studio,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
21,Port Union,Park,Yoga Studio,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
27,Eringate,Park,Yoga Studio,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
47,Richmond,Ice Cream Shop,Park,Beer Store,Field,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
54,Northwood Park,Park,Baseball Field,Yoga Studio,Empanada Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
120,Westmount,Pizza Place,Park,Yoga Studio,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
130,Kingsview Village,Park,Yoga Studio,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
173,Rosedale,Park,Playground,Bike Trail,Festival,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
177,Upper Rouge,Park,Fast Food Restaurant,Yoga Studio,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market
183,Old Mill North,Park,Metro Station,Spa,American Restaurant,River,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Exhibit


### Cluster 2 - Construction & Landscaping

In [160]:
clustered_neighborhoods.loc[clustered_neighborhoods['Cluster Labels'] == 2, clustered_neighborhoods.columns[[1] + list(range(5, clustered_neighborhoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Parkview Hill,Construction & Landscaping,Yoga Studio,Field,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


### Cluster 3 - Gyms

In [161]:
clustered_neighborhoods.loc[clustered_neighborhoods['Cluster Labels'] == 3, clustered_neighborhoods.columns[[1] + list(range(5, clustered_neighborhoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
95,Bedford Park,Gym / Fitness Center,Yoga Studio,Field,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival


### Cluster 4 - Coffee Shops, Sushi

In [162]:
clustered_neighborhoods.loc[clustered_neighborhoods['Cluster Labels'] == 4, clustered_neighborhoods.columns[[1] + list(range(5, clustered_neighborhoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Chinese Restaurant,Caribbean Restaurant,Coffee Shop,Liquor Store,Gas Station,Discount Store,Bus Line,Convenience Store,Pharmacy,Laundry Service
2,Harbourfront,Coffee Shop,Hotel,Café,Restaurant,Italian Restaurant,Pizza Place,History Museum,Music Venue,Sporting Goods Shop,Sports Bar
3,Lawrence Heights,Clothing Store,Coffee Shop,Cosmetics Shop,American Restaurant,Restaurant,Jewelry Store,Toy / Game Store,Fast Food Restaurant,Accessories Store,Bookstore
5,Queen's Park,Coffee Shop,Sandwich Place,Café,Indian Restaurant,Mediterranean Restaurant,Italian Restaurant,Middle Eastern Restaurant,Park,College Auditorium,Metro Station
7,Malvern,Pizza Place,Pharmacy,Fast Food Restaurant,Grocery Store,Skating Rink,Bubble Tea Shop,Fried Chicken Joint,Sandwich Place,Park,Empanada Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
190,The Queensway East,Gym,Eastern European Restaurant,Coffee Shop,Sushi Restaurant,Yoga Studio,Festival,Event Space,Exhibit,Falafel Restaurant,Farm
191,Royal York South East,Coffee Shop,Sushi Restaurant,Pub,Italian Restaurant,Dessert Shop,Breakfast Spot,Café,Seafood Restaurant,French Restaurant,Music Store
195,The Queensway West,BBQ Joint,Coffee Shop,American Restaurant,Restaurant,Italian Restaurant,Sporting Goods Shop,Yoga Studio,Movie Theater,Event Space,Japanese Restaurant
196,Royal York South West,Coffee Shop,Sushi Restaurant,Pub,Italian Restaurant,Dessert Shop,Breakfast Spot,Café,Seafood Restaurant,French Restaurant,Music Store
